4. Уровень кальция в крови здоровых молодых женщин равен в среднем 9.5 милиграммам на децилитр и имеет характерное стандартное отклонение 0.4 мг/дл. В сельской больнице Гватемалы для 160 здоровых беременных женщин при первом обращении для ведения беременности был измерен уровень кальция; среднее значение составило 9.57 мг/дл. Можно ли утверждать, что средний уровень кальция в этой популяции отличается от 9.5?

  Посчитайте достигаемый уровень значимости. Поскольку известны только среднее и дисперсия, а не сама выборка, нельзя использовать стандартные функции критериев — нужно реализовать формулу достигаемого уровня значимости самостоятельно.

  Округлите ответ до четырёх знаков после десятичной точки.

Используем одновобырочный Z критерий с задданой дисперсией с нулевой гипотизой о неразличимости 

In [6]:
from math import sqrt
from scipy.stats import norm

Z = (9.57-9.5) / (0.4 / sqrt(160))
p = 2*(1. - norm.cdf(abs(Z)))
round(p, 4)

0.0269

0.0269 < 0.05 => отвергаем по данным нулевую гипотезу о неразличимости уровня кальция. Но этот результат практически незначим. 9.57-9.5=0.07 << 0.4 - дисперсия в генеральной выборке

In [9]:
import numpy as np
import pandas as pd
data = pd.read_csv('diamonds.txt', sep='\t')
data.head()

,carat,depth,table,price,x,y,z
0,0.23,61.5,55.0,326,3.95,3.98,2.43
1,0.21,59.8,61.0,326,3.89,3.84,2.31
2,0.23,56.9,65.0,327,4.05,4.07,2.31
3,0.29,62.4,58.0,334,4.20,4.23,2.63
4,0.31,63.3,58.0,335,4.34,4.35,2.75


In [12]:
X = data.drop(['price'],axis=1).values
y = data['price'].values

In [13]:
from sklearn import model_selection, datasets, linear_model, metrics
train_data, test_data, train_labels, test_labels = model_selection.train_test_split(X, y, 
                                                                                    test_size = 25,
                                                                                    random_state = 1)

In [15]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

lr = LinearRegression().fit(train_data, train_labels)
rf = RandomForestRegressor(random_state=1).fit(train_data, train_labels)

In [16]:
from sklearn.metrics import mean_absolute_error as mae
print(mae(test_labels, lr.predict(test_data)))
print(mae(test_labels, rf.predict(test_data)))

755.3804166297356
711.5377980952381


In [20]:
lr_mae_scores = model_selection.cross_val_score(LinearRegression(), train_data, train_labels, scoring = 'neg_mean_absolute_error', cv = 20)

In [22]:
rf_mae_scores = model_selection.cross_val_score(RandomForestRegressor(random_state=1), train_data, train_labels, scoring = 'neg_mean_absolute_error', cv = 20, n_jobs=-1)

In [23]:
print("lr model mae: mean %.3f, std %.3f" % (lr_mae_scores.mean(), lr_mae_scores.std(ddof=1)))
print("rf model mae: mean %.3f, std %.3f" % (rf_mae_scores.mean(), rf_mae_scores.std(ddof=1)))

lr model mae: mean -889.952, std 22.808
rf model mae: mean -782.307, std 21.000


In [25]:
from statsmodels.stats.weightstats import _tconfint_generic

print("lr model mean mae 95%% confidence interval", _tconfint_generic(lr_mae_scores.mean(), lr_mae_scores.std(ddof=1),
                                                                       len(lr_mae_scores) - 1,
                                                                       0.05, 'two-sided'))

print("rf model mean mae 95%% confidence interval", _tconfint_generic(rf_mae_scores.mean(), rf_mae_scores.std(ddof=1),
                                                                         len(rf_mae_scores) - 1,
                                                                         0.05, 'two-sided'))

lr model mean mae 95%% confidence interval (-937.6899314753671, -842.2136836693776)
rf model mean mae 95%% confidence interval (-826.2608077485554, -738.3539715276681)


In [29]:
def stat_intervals(stat, alpha):
    boundaries = np.percentile(stat, [100 * alpha / 2., 100 * (1 - alpha / 2.)])
    return boundaries

mae_scors = np.abs(np.abs( (test_labels - lr.predict(test_data) )) - np.abs( (test_labels - rf.predict(test_data) )) )

alpha = 0.05
print(f"{(1.-alpha)*100}% confidence interval for mae:",  stat_intervals(mae_scors, alpha))

95.0% confidence interval for mae: [  23.68306181 1584.60079058]


In [30]:
print("mean mae 95%% confidence interval", _tconfint_generic(mae_scors.mean(), mae_scors.std(ddof=1),
                                                                         len(mae_scors) - 1,
                                                                         0.05, 'two-sided'))

rf model mean mae 95%% confidence interval (-556.1964421658438, 1361.6803928338995)


In [32]:
from statsmodels.stats.weightstats import *
one = np.abs(test_labels - lr.predict(test_data))
two = np.abs(test_labels - rf.predict(test_data))
print("95%% confidence interval: [%f, %f]" % DescrStatsW(np.abs(one - two)).tconfint_mean())

95% confidence interval: [210.954292, 594.529659]


# Тест: Параметрические критерии

4., 5.

In [41]:
df = pd.read_csv("banknotes.txt", sep='\t')
X = df.drop(['real'], axis=1).values
y = df['real'].values

In [49]:
train_data, test_data, train_labels, test_labels = model_selection.train_test_split(X, y, 
                                                                                    test_size = 0.25,
                                                                                    random_state = 1)

In [52]:
df

,X1,X2,X3,X4,X5,X6,real
0,214.8,131.0,131.1,9.0,9.7,141.0,1
1,214.6,129.7,129.7,8.1,9.5,141.7,1
2,214.8,129.7,129.7,8.7,9.6,142.2,1
3,214.8,129.7,129.6,7.5,10.4,142.0,1
4,215.0,129.6,129.7,10.4,7.7,141.8,1
...,...,...,...,...,...,...,...
195,215.0,130.4,130.3,9.9,12.1,139.6,0
196,215.1,130.3,129.9,10.3,11.5,139.7,0
197,214.8,130.3,130.4,10.6,11.1,140.0,0
198,214.7,130.7,130.8,11.2,11.2,139.4,0


In [130]:
from sklearn.linear_model import LogisticRegression
lr1 = LogisticRegression(solver='liblinear', multi_class='ovr').fit(train_data[:, [0, 1, 2]], train_labels)
lr2 = LogisticRegression(solver='liblinear', multi_class='ovr').fit(train_data[:, [3, 4, 5]], train_labels)

In [131]:
lr2

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='ovr', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [132]:
print(len(np.where(test_labels == lr1.predict(test_data[:, [0, 1, 2]]))[0]) / len(test_labels))
print(len(np.where(test_labels == lr2.predict(test_data[:, [3, 4, 5]]))[0]) / len(test_labels))

0.8
0.98


In [137]:
a = test_labels != lr1.predict(test_data[:, [0, 1, 2]])
b = test_labels != lr2.predict(test_data[:, [3, 4, 5]])

In [138]:
def proportions_diff_confint_rel(sample1, sample2, alpha = 0.05):
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
    sample = list(zip(sample1, sample2))
    n = len(sample)
        
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    left_boundary = float(f - g) / n  - z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    right_boundary = float(f - g) / n  + z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    return (left_boundary, right_boundary)


def proportions_diff_z_stat_rel(sample1, sample2):
    sample = list(zip(sample1, sample2))
    n = len(sample)
    
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    return float(f - g) / np.sqrt(f + g - float((f - g)**2) / n )

def proportions_diff_z_test(z_stat, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - scipy.stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return scipy.stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - scipy.stats.norm.cdf(z_stat)
    
print("p-value: %f" % proportions_diff_z_test(proportions_diff_z_stat_rel(a, b)))

p-value: 0.003297


In [142]:
print("95%% confidence interval for a difference between proportions: [%.4f, %f]" %\
      proportions_diff_confint_rel(a, b))

95% confidence interval for a difference between proportions: [0.0599, 0.300055]


6., 7.

In [84]:
one = 541.4
two = 525
Z = (one - two) / 10

p = (1. - norm.cdf(Z))
round(p, 4)

0.0505

In [83]:
one = 541.5
two = 525
Z = (one - two) / 10

p = (1. - norm.cdf(Z))
round(p, 4)

0.0495

3.

In [89]:
def proportions_diff_z_stat_ind(sample1, sample2):
    n1 = len(sample1)
    n2 = len(sample2)
    
    p1 = float(sum(sample1)) / n1
    p2 = float(sum(sample2)) / n2 
    P = float(p1*n1 + p2*n2) / (n1 + n2)
    
    return (p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))

def proportions_diff_z_test(z_stat, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - scipy.stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return scipy.stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - scipy.stats.norm.cdf(z_stat)
    
import scipy 
def proportions_diff_confint_ind(sample1, sample2, alpha = 0.05):    
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
    
    p1 = float(sum(sample1)) / len(sample1)
    p2 = float(sum(sample2)) / len(sample2)
    
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    
    return (left_boundary, right_boundary)

In [79]:
4/16 - 10/34

-0.04411764705882354

In [99]:
a = [1]*4 + [0]*12
b = [1]*10 + [0]*24

In [100]:
print("p-value: %f" % proportions_diff_z_test(proportions_diff_z_stat_ind(b, a), 'greater'))

p-value: 0.372930
